In [2]:
import pandas as pd

In [3]:
data = pd.read_excel('MORE_DB_date.xlsx')

In [5]:
data.head()

,Артикул,Наименоване товара (цена за 1 кг/л),Страна,"Ставка НДС\nрозн, %",Кол-во,Отпускная цена,Опт. надб.,Торго-\nвая на-\nценка,Ставка НДС поставщика,Розн. цена с НДС,Розничная сумма с НДС,поставщик,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,1005559,Леденцы Орбит лимон и мята 35г (4.49 100 г),Россия,20,8,"0,9",0,45.37 (45.00),20,1.57,12.56,АЛИДИ-Вест ИООО,04.02.2020,NaN,NaN
1,1001403,Драже М& М с арахисом 45г (3.36 100 г),Россия,20,32,"0,87",0,44.64 (45.00),20,1.51,48.32,АЛИДИ-Вест ИООО,04.02.2020,NaN,NaN
2,1001404,Батончик Милки Вэй 26г (2.54 100 г),Россия,20,108,"0,38",0,44.74 (45.00),20,0.66,71.28,АЛИДИ-Вест ИООО,04.02.2020,NaN,NaN
3,1001395,Батончик Марс 50г (24.4 кг),Россия,20,36,"0,7",0,45.24 (45.00),20,1.22,43.92,АЛИДИ-Вест ИООО,04.02.2020,NaN,NaN
4,1001392,Батончик Сникерс 95г (19.37 кг),Россия,20,32,"1,18",0,29.94 (30.00),20,1.84,58.88,АЛИДИ-Вест ИООО,04.02.2020,NaN,NaN


In [4]:
import psycopg2

In [5]:
conn = psycopg2.connect(dbname='more', user='postgres', 
                        password='1458', host='localhost')

In [44]:
cursor = conn.cursor()

In [9]:
conn.rollback()

In [53]:

with conn.cursor() as cursor:
    conn.autocommit = True
    for country in data['Страна'].unique():
        query = f'INSERT INTO more_table.country (name) VALUES (\'{country}\');'
        cursor.execute(query)
    

In [55]:
with conn.cursor() as cursor:
    conn.autocommit = True
    for prod in data['Наименоване товара (цена за 1 кг/л)'].unique()[:20]:
        query = f'INSERT INTO more_table.product (name, unit) VALUES (\'{prod}\', \'unit\');'
        cursor.execute(query)

In [58]:
with conn.cursor() as cursor:
    conn.autocommit = True
    for sap in data['поставщик'].unique():
        query = f'INSERT INTO more_table.saplier (name, email, tel) VALUES (\'{sap}\', \'{sap}@gmail.com\', \'+34241241\');'
        cursor.execute(query)

In [30]:
with conn.cursor() as cursor:
    conn.autocommit = True
    for row in range(20):
        prod_name = data.loc[row, 'Наименоване товара (цена за 1 кг/л)']
        c_name = data.loc[row, 'Страна']
        sup_name = data.loc[row, 'поставщик']
        u_price = data.loc[row, 'Розн. цена с НДС']
        quant = data.loc[row, 'Кол-во']
        query = f'SELECT  more_table.insert_deliver_warehouse(\'{prod_name}\'::text,\'{c_name}\'::text, \'{sup_name}\'::text, {u_price}::float, {quant}::float);'
        cursor.execute(query)

ForeignKeyViolation: insert or update on table "delivery" violates foreign key constraint "delivery_idproduct_fkey"
DETAIL:  Key (idproduct)=(22) is not present in table "product".
CONTEXT:  SQL statement "INSERT INTO more_table.delivery (idproduct, idcountry, idsumplier, unit_price, quantity)
        VALUES (
                 id_prod,
                 (SELECT idcountry FROM  more_table.country WHERE name=c_name),
                 (SELECT idsaplier FROM  more_table.saplier WHERE name=sup_name),
                 u_price,
                 quant
                )"
PL/pgSQL function more_table.insert_deliver_warehouse(text,text,text,double precision,double precision) line 6 at SQL statement


In [10]:
with conn.cursor() as cursor:
    cursor.execute('SELECT * FROM more_table.country;')
    country = [{'id':row[0], 'country':row[1]} for row in cursor.fetchall()]

In [1]:
from more_db import More_DB_commnad

In [2]:
more_db = More_DB_commnad('more','postgres', '1458', 'localhost')

In [3]:
u''.join().encode('utf-8').decode()

TypeError: str.join() takes exactly one argument (0 given)

In [3]:
more_db.send_email('dad','Драже М& М с арахисом 45г (3.36 100 г)')

Sended email ot sapplier


In [4]:
more_db.insert_sales('Драже М& М с арахисом 45г (3.36 100 г)', 32)

0

In [3]:
sup = more_db.select_warehouse()

In [1]:
from fastapi import Form

In [12]:
for row in sup:
    print(row['id'], row['name'], row['email'],row['tel'], sep='\t')

1	АЛИДИ-Вест ИООО	АЛИДИ-Вест ИООО@gmail.com	+34241241
2	ДанонБел ИООО	ДанонБел ИООО@gmail.com	+34241241
3	Кока-Кола Бевриджиз Белоруссия УП	Кока-Кола Бевриджиз Белоруссия УП@gmail.com	+34241241
4	Савушкин продукт ОАО	Савушкин продукт ОАО@gmail.com	+34241241
5	Табак-инвест ООО	Табак-инвест ООО@gmail.com	+34241241
6	Алиди-Вест филиал №1 НеСТ	Алиди-Вест филиал №1 НеСТ@gmail.com	+34241241
7	Бритиш-Американ Тобакко Компани	Бритиш-Американ Тобакко Компани@gmail.com	+34241241
8	Мостра-групп Кофе ОДО	Мостра-групп Кофе ОДО@gmail.com	+34241241
9	ООО ЭНЕРГИЯ	ООО ЭНЕРГИЯ@gmail.com	+34241241
10	Диакония УП	Диакония УП@gmail.com	+34241241
11	Мостра Групп ОДО	Мостра Групп ОДО@gmail.com	+34241241
12	Продуктория ООО	Продуктория ООО@gmail.com	+34241241
13	Туссон плюс ООО	Туссон плюс ООО@gmail.com	+34241241
14	Маркетинговая Инновационная Группа ООО	Маркетинговая Инновационная Группа ООО@gmail.com	+34241241
15	Корпорация НОЯКС ООО	Корпорация НОЯКС ООО@gmail.com	+34241241
16	Шедевры вкуса ООО	Шедевры вкуса

In [4]:
import sys

In [8]:
sys.path.append('/home/uladzimir/MORE/MORE_FAST_API/model')

In [9]:
sys.path

['/home/uladzimir/MORE/MORE_FAST_API/model',
 '/home/uladzimir/.vscode/extensions/ms-toolsai.jupyter-2021.11.1001550889/pythonFiles',
 '/home/uladzimir/.vscode/extensions/ms-toolsai.jupyter-2021.11.1001550889/pythonFiles/lib/python',
 '/home/uladzimir/anaconda3/envs/more/lib/python310.zip',
 '/home/uladzimir/anaconda3/envs/more/lib/python3.10',
 '/home/uladzimir/anaconda3/envs/more/lib/python3.10/lib-dynload',
 '',
 '/home/uladzimir/anaconda3/envs/more/lib/python3.10/site-packages',
 '/home/uladzimir/anaconda3/envs/more/lib/python3.10/site-packages/IPython/extensions',
 '/home/uladzimir/.ipython',
 '/',
 '/home/uladzimir/MORE/MORE_FAST_API/model']

In [7]:
import smtplib

In [8]:
smtpObj = smtplib.SMTP('smtp.mail.ru', 587)

In [9]:
smtpObj.starttls()

(220, b'2.0.0 Start TLS')

(235, b'Authentication succeeded')

In [11]:
smtpObj.sendmail('krutkovova09gmail.com@mail.ru', 'krutkovova24@gmail.com', 'Hellow')

{}